# A Chatbot for the Climate Policy of Vienna #
With this Capstone Project, I have build a Large Language Model out of the key climate policy documents of Vienna since the Introduction of the Climate Protection Policy in 1999.
The Climate Protection Policies 1 and 2 (KliP 1 & 2 - short for Klimaschutzprogramme, you can find more information here: https://www.wien.gv.at/umwelt/programm-klip) were introduced in 1999 and ran until 2021. 
In 2020, the coalition government (ÖVP & the Green Party) announced the commitment to achieving climate neutrality by 2040 and shortly after this, in 2022, the Vienna Climate Roadmap (https://www.wien.gv.at/spezial/klimafahrplan/) was introduced. 
With these 4 documents, spanning approximately 700 pages, a Chatbot was created. 

In [35]:
import os
import openai
import shutil
import warnings
import pandas as pd

from llama_index.core import (
    SimpleDirectoryReader, ListIndex, VectorStoreIndex, TreeIndex,
    KeywordTableIndex, SimpleKeywordTableIndex, DocumentSummaryIndex,
    KnowledgeGraphIndex, StorageContext, load_index_from_storage
)
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import ChatPromptTemplate, ChatMessage, MessageRole
from llama_index.core.response_synthesizers import ResponseMode

In [36]:
openai.api_key = os.environ["OPENAI_API_KEY"]
# Suppressing warnings
warnings.filterwarnings('ignore')
embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
llm = OpenAI(temperature=0, model="gpt-3.5-turbo", presence_penalty=-2, top_p=1)

In [37]:
# Directory paths
DOCS_DIR = "/Users/sophiehamann/Documents/MA_Universität_Wien/SoSe24/GenAI/chatbot_Vienna/data"
INDEX_DIR = "/Users/sophiehamann/Documents/MA_Universität_Wien/SoSe24/GenAI/chatbot_Vienna/data/Index/Intro"
PERSIST_DIR = "/Users/sophiehamann/Documents/MA_Universität_Wien/SoSe24/GenAI/chatbot_Vienna/data/Index/ChatExample"

In [38]:
# Ensure the directories exist
os.makedirs(DOCS_DIR, exist_ok=True)
os.makedirs(INDEX_DIR, exist_ok=True)

In [39]:
# Load documents
pdf_files = [os.path.join(DOCS_DIR, filename) for filename in os.listdir(DOCS_DIR) if filename.endswith('.pdf')]
documents = SimpleDirectoryReader(input_files=pdf_files).load_data()

In [40]:
# Create index
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model, llm=llm)
# Save the index to storage
index.storage_context.persist(persist_dir=INDEX_DIR)
# Load index from storage if it exists
if os.path.exists(INDEX_DIR):
    storage_context = StorageContext.from_defaults(persist_dir=INDEX_DIR)
    index = load_index_from_storage(storage_context)

In [41]:
# Define query engine
query_engine = index.as_query_engine()

In [42]:
# Example Queries
print(query_engine.query("What is the document about?"))
print(query_engine.query("What the most important point mentioned in the document?"))

The document is about the KliP-Wien climate protection program, providing an overview of its basis and approach.
The most important point mentioned in the document is the emphasis on strengthening democracy in Europe, with Vienna aiming to lead in implementing democratic values and becoming a best practice for democracy in Europe.


In [43]:
# Prepare the persistent directory
if os.path.exists(PERSIST_DIR):
    shutil.rmtree(PERSIST_DIR)
os.makedirs(PERSIST_DIR)

In [44]:
# Define advanced query engine settings
query_engine = index.as_query_engine(similarity_top_k=3, retriever_mode="embedding", response_mode="compact", verbose=True)
print(query_engine.query("What is KLiP?"))

KLiP is a comprehensive program in Vienna that serves as both a climate protection and economic initiative. It aims to reduce greenhouse gas emissions, improve the overall environmental and living conditions in Vienna, enhance Vienna's reputation as an environmental model city, strengthen the city's economy, and create and secure jobs.


In [45]:
# Chat engine setup
TEXT_QA_SYSTEM_PROMPT = ChatMessage(
    content=(
        "You are an expert Q&A system that is trusted in Vienna.\n"
        "Always answer the query using the provided context information, "
        "and not prior knowledge.\n"
        "Some rules to follow:\n"
        "1. Never directly reference the given context in your answer.\n"
        "2. Avoid statements like 'Based on the context, ...' or "
        "'The context information ...' or anything along "
        "those lines."
    ),
    role=MessageRole.SYSTEM,
)

TEXT_QA_PROMPT_TMPL_MSGS = [
    TEXT_QA_SYSTEM_PROMPT,
    ChatMessage(
        content=(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the query.\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
        role=MessageRole.USER,
    ),
]

CHAT_REFINE_PROMPT_TMPL_MSGS = [
    ChatMessage(
        content=(
            "You are an expert Q&A system that strictly operates in two modes "
            "when refining existing answers:\n"
            "1. **Rewrite** an original answer using the new context.\n"
            "2. **Repeat** the original answer if the new context isn't useful.\n"
            "Never reference the original answer or context directly in your answer.\n"
            "If the query is unrelated to the context, just answer: I don't know. \n"
            "When in doubt, just repeat the original answer.\n"
            "New Context: {context_msg}\n"
            "Query: {query_str}\n"
            "Original Answer: {existing_answer}\n"
            "New Answer: "
        ),
        role=MessageRole.USER,
    )
]

CHAT_TEXT_QA_PROMPT = ChatPromptTemplate(message_templates=TEXT_QA_PROMPT_TMPL_MSGS)
CHAT_REFINE_PROMPT = ChatPromptTemplate(message_templates=CHAT_REFINE_PROMPT_TMPL_MSGS)

In [46]:
chat_engine = index.as_chat_engine(chat_mode="context", verbose=True, text_qa_template=CHAT_REFINE_PROMPT)
chat_engine.reset()
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  Tell me about the goals for Climate Neutrality. 


Assistant: The goals for Climate Neutrality in Vienna are ambitious and comprehensive. The city of Vienna aims to achieve climate neutrality by 2040, which means reducing greenhouse gas emissions to zero or near-zero levels. This goal is part of the city's efforts to combat climate change and create a sustainable future for its residents.

Some of the key priorities and strategies outlined to achieve climate neutrality in Vienna include:

1. Reducing emissions from fossil-fuel-powered vehicles and gas heating systems by transitioning to electric vehicles, promoting alternative modes of transportation, and improving public transportation infrastructure.
   
2. Transitioning away from gas in heating systems by reducing building energy consumption, switching to district heating systems, and utilizing ambient heat with electric heat pumps.

3. Decarbonizing the city's electricity and district heating systems by expanding renewable energy sources in Vienna and Austria. Green gas, which refe

Human:  exit


In [47]:
# Adding Memory and System Prompt
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=80000)
chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
            "Context information from multiple sources is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the information from multiple sources"
            "answer the query.\n"
            "If the query is unrelated to the context, just answer: I don't know"
            "Always start your answer with 'Dear Student'" 
            "Query: {query_str}\n"
            "Answer: "
    ),
)

In [48]:
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  Is it possible to achieve Climate Neutrality in Vienna until 2040?


Assistant: Dear Student
Based on the information provided in the sources, it is indeed possible to achieve climate neutrality in Vienna by 2040. Both the EU directives and the coalition agreement in Vienna emphasize the commitment to becoming climate neutral by 2040. Vienna has already taken significant steps towards this goal, such as implementing fossil-free heating, improving public transportation, promoting renewable energy for vehicles, and increasing green spaces in the city. The city has a comprehensive strategy in place, including the establishment of a Wiener Klimaschutzgesetz (Vienna Climate Protection Law) and a Klimabudget (Climate Budget) to ensure efficient and effective implementation of climate protection measures. Therefore, with the continued dedication and implementation of these initiatives, Vienna is on track to achieve climate neutrality by 2040.




Human:  exit


In [49]:
memory.to_dict()

{'chat_store': {'store': {'chat_history': [{'role': <MessageRole.USER: 'user'>,
     'content': 'Is it possible to achieve Climate Neutrality in Vienna until 2040?',
     'additional_kwargs': {}},
    {'role': <MessageRole.ASSISTANT: 'assistant'>,
     'content': 'Dear Student\nBased on the information provided in the sources, it is indeed possible to achieve climate neutrality in Vienna by 2040. Both the EU directives and the coalition agreement in Vienna emphasize the commitment to becoming climate neutral by 2040. Vienna has already taken significant steps towards this goal, such as implementing fossil-free heating, improving public transportation, promoting renewable energy for vehicles, and increasing green spaces in the city. The city has a comprehensive strategy in place, including the establishment of a Wiener Klimaschutzgesetz (Vienna Climate Protection Law) and a Klimabudget (Climate Budget) to ensure efficient and effective implementation of climate protection measures. Ther